In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
red_quality_df = pd.read_csv("red_wine_quality.csv")
red_quality_df.head()

,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulfates,alcohol,quality
0,red,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,red,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,red,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,red,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,red,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
# Drop the non-beneficial ID columns, 'residual_sugar', 'free_sulfur_dioxide', and 'ph'.
red_quality_df.drop(['residual_sugar', 'free_sulfur_dioxide', 'ph'], axis=1, inplace=True)

In [7]:
# Create Classification version of target variable
# Idea for classification inspired by Terence Shin on 
# towardsdatascience.com/predicting-wine-quality-with-several-classification-techniques-179038ea6434
red_quality_df['goodquality'] = [1 if x >= 7 else 0 for x in red_quality_df['quality']]

In [16]:
red_quality_df

,type,fixed_acidity,volatile_acidity,citric_acid,chlorides,total_sulfur_dioxide,density,sulfates,alcohol,quality,goodquality
0,red,7.4,0.700,0.00,0.076,34.0,0.99780,0.56,9.4,5,0
1,red,7.8,0.880,0.00,0.098,67.0,0.99680,0.68,9.8,5,0
2,red,7.8,0.760,0.04,0.092,54.0,0.99700,0.65,9.8,5,0
3,red,11.2,0.280,0.56,0.075,60.0,0.99800,0.58,9.8,6,0
4,red,7.4,0.700,0.00,0.076,34.0,0.99780,0.56,9.4,5,0
...,...,...,...,...,...,...,...,...,...,...,...
1588,red,6.8,0.620,0.08,0.068,38.0,0.99651,0.82,9.5,6,0
1589,red,6.2,0.600,0.08,0.090,44.0,0.99490,0.58,10.5,5,0
1590,red,6.3,0.510,0.13,0.076,40.0,0.99574,0.75,11.0,6,0
1591,red,5.9,0.645,0.12,0.075,44.0,0.99547,0.71,10.2,5,0


In [10]:
# Split our preprocessed data into our features and target arrays
y= red_quality_df["goodquality"].values
X = red_quality_df.drop(columns=["goodquality", 'type'], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
number_hidden_nodes_layer1 =  50
number_hidden_nodes_layer2 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=number_hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=number_hidden_nodes_layer2, activation="relu", input_dim=number_input_features))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                500       
                                                                 
 dense_1 (Dense)             (None, 25)                1275      
                                                                 
 dense_2 (Dense)             (None, 1)                 26        
                                                                 
Total params: 1,801
Trainable params: 1,801
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
38/38 [==============================] - 3s 6ms/step - loss: 0.5264 - accuracy: 0.7672
Epoch 2/100
38/38 [==============================] - 0s 7ms/step - loss: 0.2720 - accuracy: 0.9271
Epoch 3/100
38/38 [==============================] - 0s 10ms/step - loss: 0.1729 - accuracy: 0.9648
Epoch 4/100
38/38 [==============================] - 0s 7ms/step - loss: 0.1210 - accuracy: 0.9841
Epoch 5/100
37/38 [============================>.] - ETA: 0s - loss: 0.0837 - accuracy: 0.9907
Epoch 00005: saving model to checkpoints\weights.05.hdf5
38/38 [==============================] - 0s 12ms/step - loss: 0.0832 - accuracy: 0.9908
Epoch 6/100
38/38 [==============================] - 0s 6ms/step - loss: 0.0567 - accuracy: 0.9916
Epoch 7/100
38/38 [==============================] - 0s 6ms/step - loss: 0.0387 - accuracy: 0.9950
Epoch 8/100
38/38 [==============================] - 0s 7ms/step - loss: 0.0274 - accuracy: 0.9950
Epoch 9/100
38/38 [==============================] - 0s 10ms/step 

38/38 [==============================] - 0s 7ms/step - loss: 1.3242e-04 - accuracy: 1.0000
Epoch 62/100
38/38 [==============================] - 0s 6ms/step - loss: 1.2665e-04 - accuracy: 1.0000
Epoch 63/100
38/38 [==============================] - 0s 6ms/step - loss: 1.2184e-04 - accuracy: 1.0000
Epoch 64/100
38/38 [==============================] - 0s 7ms/step - loss: 1.1697e-04 - accuracy: 1.0000: 0s - loss: 1.2509e-04 - accuracy: 1.
Epoch 65/100
37/38 [============================>.] - ETA: 0s - loss: 1.1319e-04 - accuracy: 1.0000
Epoch 00065: saving model to checkpoints\weights.65.hdf5
38/38 [==============================] - 0s 10ms/step - loss: 1.1243e-04 - accuracy: 1.0000
Epoch 66/100
38/38 [==============================] - 0s 5ms/step - loss: 1.0790e-04 - accuracy: 1.0000
Epoch 67/100
38/38 [==============================] - 0s 6ms/step - loss: 1.0377e-04 - accuracy: 1.0000
Epoch 68/100
38/38 [==============================] - 0s 5ms/step - loss: 9.9954e-05 - accuracy: 1.000